In [1]:
import pandas as pd
import pathlib
import glob
import os

## Check if trackviews are complete

In [2]:
col_desc = pd.read_excel('../../data/column_descriptors_standardized.xlsx')

In [3]:
col_desc.head()

,column_ID,wig_files,control,experimental,column_ID_2,num_replicates_control,num_replicates_experimental,meaning,year,paper_title,...,in_vitro_media,carbon_source,stress_description,GI_RvID,GI_name,MicArr_or_TnSeq,stat_analysis,mouse_strain,cell_type,Mtb_strain
0,2011_Griffin_2,True,griffin_glycerol,griffin_cholesterol,griffin_cholesterol_vs_griffin_glycerol,2,3,Mutants exhiting altered fitness during choles...,2011.0,High-Resolution Phenotypic Profiling Defines G...,...,minimal_media,cholesterol,-,NaN,NaN,TnSeq,NaN,NaN,NaN,H37Rv
1,2013_DeJesus,True,mbio_H37Rv,griffin_glycerol,griffin_glycerol_vs_mbio_H37Rv,14,2,NaN,2013.0,A Hidden Markov Model for identifying essentia...,...,NaN,glycerol,-,NaN,NaN,TnSeq,NaN,NaN,NaN,NaN
2,2013_Zhang_1A,True,zhang_input_library,zhang_wt_mouse_d10,zhang_wt_mouse_d10_vs_zhang_input_library,1,3,Mutants exhibiting altered fitness in mouse in...,2013.0,Tryptophan biosynthesis protects mycobacteria ...,...,-,-,NaN,NaN,NaN,TnSeq,NaN,WT,NaN,NaN
3,2013_Zhang_1B,True,zhang_input_library,zhang_wt_mouse_d45,zhang_wt_mouse_d45_vs_zhang_input_library,1,3,Mutants exhibiting altered fitness in mouse in...,2013.0,Tryptophan biosynthesis protects mycobacteria ...,...,-,-,NaN,NaN,NaN,TnSeq,NaN,WT,NaN,NaN
4,2013_Zhang_2A,True,zhang_wt_mouse_d10,zhang_mhcii_mouse_d10,zhang_mhcii_mouse_d10_vs_zhang_wt_mouse_d10,3,4,Mutants exhibiting altered fitness in immunode...,2013.0,Tryptophan biosynthesis protects mycobacteria ...,...,-,-,NaN,NaN,NaN,TnSeq,NaN,MHCII−/−,NaN,NaN


In [4]:
# check if trackviews exist
trackview_folders=list(pathlib.Path(r'C:\Users\light\Dropbox (WCMC Schnappinger)\tuberculosis\tuberculosis\TrackView').glob('*'))
trackview_names=[os.path.basename(x) for x in trackview_folders]

In [5]:
all_conditions=list(col_desc['control']) + list(col_desc['experimental'])

In [6]:
set(all_conditions).difference(set(trackview_names))

{'nambi_2015_ctpC',
 'nambi_2015_wt',
 'xu_emb_0',
 'xu_emb_2.5',
 'xu_emb_3',
 'xu_inh_0',
 'xu_inh_02',
 'xu_inh_025',
 'xu_mero_0',
 'xu_mero_2.5',
 'xu_rif_0',
 'xu_rif_4',
 'xu_van_0',
 'xu_van_16'}

## lfc

In [57]:
lfc=pd.read_csv('../../data/standardized_data/result_logfc_matrix_2019_12_10.csv')

In [58]:
qval=pd.read_csv('../../data/standardized_data/result_qval_matrix_2019_12_10.csv')

In [59]:
gene_names= pd.read_excel('../../data/annotations/DeJesus_mbio.xlsx', header=1)
gene_names.head()

,ORF ID,Name,Description,Number of TA Sites,Number of Permissive (P) Sites,Number of Non-Permissive (NP) Sites,Number of Sites Belonging to Essential State,Number of Sites Belonging to Growth-Defect State,Number of Sites Belonging to Non-Essential State,Number of Sites Belonging to Growth-Advantage State,Fraction of Sites with Insertions,Mean Normalized Read-Count At Non-Zero Sites,Final Call
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,32,29,3,31,0,1,0,0.125000,45.000000,ES
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,31,27,4,31,0,0,0,0.032258,1.000000,ES
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,35,32,3,0,0,35,0,0.857143,38.033333,NE
3,Rv0004,-,hypothetical protein,8,8,0,0,0,8,0,0.375000,144.000000,NE
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,40,36,4,39,0,1,0,0.025000,26.000000,ES


In [60]:
gene_names=gene_names.rename(columns={'Name': 'gene_name', 'ORF ID': 'Rv_ID'})

In [61]:
lfc.head()

,Unnamed: 0,PE35_KO_vs_mbio_H37Rv,PPE68_KO_vs_mbio_H37Rv,Rv0950c_KO_vs_CB_WT,Rv0954_KO_vs_RJ_WT,Rv1096_KO_vs_CB_WT,Rv3005c_KO_day32_vs_dejesus_H37Rv_day32,Rv3594_KO_vs_Rubin_FLUTE_WT,Rv3684_KO_vs_CB_WT,Rv3717_KO_vs_Rubin_FLUTE_WT,...,zhang_AA_Rescue_vs_zhang_in_vitro_control_Rescue,zhang_DETA-NO_pH_7.0_vs_zhang_pH_7.0_no_NO_control,zhang_Fe_1.5mM_vs_zhang_Fe_450uM,zhang_Trp_Rescue_vs_zhang_in_vitro_control_Rescue,zhang_Tyloxapol_pH_6.5_vs_zhang_Tyloxapol_pH_4.5,zhang_Tyloxapol_pH_6.5_vs_zhang_pcit_pH_4.5,zhang_mhcii_mouse_d10_vs_zhang_wt_mouse_d10,zhang_mhcii_mouse_d45_vs_zhang_wt_mouse_d45,zhang_wt_mouse_d10_vs_zhang_input_library,zhang_wt_mouse_d45_vs_zhang_input_library
0,Rv0001,-0.41,0.01,0.00,0.00,0.00,3.12,-0.06,0.00,-0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Rv0002,3.28,2.38,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Rv0003,-0.03,-0.08,0.03,-1.20,0.39,3.83,0.31,0.17,-0.62,...,-2.93,-0.31,1.09,-2.13,0.51,0.41,1.30,-0.49,-1.26,-2.25
3,Rv0004,-3.81,-3.39,-2.60,-6.83,1.54,0.00,1.79,1.08,4.91,...,-3.56,0.00,1.96,-3.56,0.00,0.00,0.00,0.70,-2.62,-2.62
4,Rv0005,-0.22,0.14,0.13,-0.08,0.27,-2.38,0.70,-0.71,-0.37,...,-1.47,-0.89,-2.36,-1.89,2.90,2.20,-4.05,0.30,0.69,-3.55


In [62]:
lfc=lfc.rename(columns={'Unnamed: 0': 'Rv_ID'})
qval=qval.rename(columns={'Unnamed: 0': 'Rv_ID'})

In [63]:
lfc=lfc.merge(gene_names[['Rv_ID', 'gene_name', 'Description']], how='left')

In [64]:
lfc = lfc.melt(id_vars=['Rv_ID', 'gene_name', 'Description'], var_name='Expt', value_name='log2FC')
qval = qval.melt(id_vars=['Rv_ID'], var_name='Expt', value_name='q-val')

In [65]:
print (lfc.shape, qval.shape)

(259520, 5) (259520, 3)


In [66]:
main_data = lfc.merge(qval, how='left', on=['Rv_ID', 'Expt'])

In [67]:
print(main_data.shape)

(259520, 6)


In [68]:
main_data

,Rv_ID,gene_name,Description,Expt,log2FC,q-val
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,PE35_KO_vs_mbio_H37Rv,-0.41,1.00000
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,PE35_KO_vs_mbio_H37Rv,3.28,1.00000
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,PE35_KO_vs_mbio_H37Rv,-0.03,1.00000
3,Rv0004,-,hypothetical protein,PE35_KO_vs_mbio_H37Rv,-3.81,1.00000
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,PE35_KO_vs_mbio_H37Rv,-0.22,1.00000
...,...,...,...,...,...,...
259515,Rv3921c,-,Probable conserved transmembrane protein,zhang_wt_mouse_d45_vs_zhang_input_library,0.00,1.00000
259516,Rv3922c,-,Possible hemolysin,zhang_wt_mouse_d45_vs_zhang_input_library,-2.06,0.32788
259517,Rv3923c,rnpA,Ribonuclease P protein component RnpA (RNaseP ...,zhang_wt_mouse_d45_vs_zhang_input_library,-3.80,0.81749
259518,Rv3924c,rpmH,50S ribosomal protein L34 RpmH,zhang_wt_mouse_d45_vs_zhang_input_library,0.00,1.00000


In [69]:
# main_data.to_csv('../../data/main_data_dash.csv')

## Make expt metadata

In [70]:
col_desc = pd.read_excel('../../data/column_descriptors_standardized.xlsx')

In [71]:
col_desc.head()

,column_ID,wig_files,control,experimental,column_ID_2,num_replicates_control,num_replicates_experimental,meaning,year,paper_title,...,in_vitro_media,carbon_source,stress_description,GI_RvID,GI_name,MicArr_or_TnSeq,stat_analysis,mouse_strain,cell_type,Mtb_strain
0,2011_Griffin_2,True,griffin_glycerol,griffin_cholesterol,griffin_cholesterol_vs_griffin_glycerol,2,3,Mutants exhiting altered fitness during choles...,2011.0,High-Resolution Phenotypic Profiling Defines G...,...,minimal_media,cholesterol,-,NaN,NaN,TnSeq,NaN,NaN,NaN,H37Rv
1,2013_DeJesus,True,mbio_H37Rv,griffin_glycerol,griffin_glycerol_vs_mbio_H37Rv,14,2,NaN,2013.0,A Hidden Markov Model for identifying essentia...,...,NaN,glycerol,-,NaN,NaN,TnSeq,NaN,NaN,NaN,NaN
2,2013_Zhang_1A,True,zhang_input_library,zhang_wt_mouse_d10,zhang_wt_mouse_d10_vs_zhang_input_library,1,3,Mutants exhibiting altered fitness in mouse in...,2013.0,Tryptophan biosynthesis protects mycobacteria ...,...,-,-,NaN,NaN,NaN,TnSeq,NaN,WT,NaN,NaN
3,2013_Zhang_1B,True,zhang_input_library,zhang_wt_mouse_d45,zhang_wt_mouse_d45_vs_zhang_input_library,1,3,Mutants exhibiting altered fitness in mouse in...,2013.0,Tryptophan biosynthesis protects mycobacteria ...,...,-,-,NaN,NaN,NaN,TnSeq,NaN,WT,NaN,NaN
4,2013_Zhang_2A,True,zhang_wt_mouse_d10,zhang_mhcii_mouse_d10,zhang_mhcii_mouse_d10_vs_zhang_wt_mouse_d10,3,4,Mutants exhibiting altered fitness in immunode...,2013.0,Tryptophan biosynthesis protects mycobacteria ...,...,-,-,NaN,NaN,NaN,TnSeq,NaN,MHCII−/−,NaN,NaN


In [72]:
col_desc.columns

Index(['column_ID', 'wig_files', 'control', 'experimental', 'column_ID_2',
       'num_replicates_control', 'num_replicates_experimental', 'meaning',
       'year', 'paper_title', 'paper_URL', 'journal', 'first_author',
       'last_author', 'in_vitro_cell_vivo', 'in_vitro_media', 'carbon_source',
       'stress_description', 'GI_RvID', 'GI_name', 'MicArr_or_TnSeq',
       'stat_analysis', 'mouse_strain', 'cell_type', 'Mtb_strain'],
      dtype='object')

In [73]:
col_desc=col_desc.drop(columns=['column_ID', 'wig_files', 'control', 'experimental','year', 'journal', 'first_author', 'last_author',  'in_vitro_cell_vivo', 'in_vitro_media', 'carbon_source',
       'stress_description', 'GI_RvID', 'GI_name', 'MicArr_or_TnSeq',
       'stat_analysis', 'mouse_strain', 'cell_type', 'Mtb_strain'])

In [74]:
main_data = main_data.merge(col_desc, left_on='Expt', right_on='column_ID_2', how='left')

In [75]:
main_data.shape

(259520, 12)

In [76]:
main_data=main_data.drop(columns='column_ID_2')

In [77]:
main_data

,Rv_ID,gene_name,Description,Expt,log2FC,q-val,num_replicates_control,num_replicates_experimental,meaning,paper_title,paper_URL
0,Rv0001,dnaA,Chromosomal replication initiator protein DnaA,PE35_KO_vs_mbio_H37Rv,-0.41,1.00000,14,1,Mutants exhibiting altered fitness in the abse...,NaN,NaN
1,Rv0002,dnaN,DNA polymerase III (beta chain) DnaN (DNA nucl...,PE35_KO_vs_mbio_H37Rv,3.28,1.00000,14,1,Mutants exhibiting altered fitness in the abse...,NaN,NaN
2,Rv0003,recF,DNA replication and repair protein RecF (singl...,PE35_KO_vs_mbio_H37Rv,-0.03,1.00000,14,1,Mutants exhibiting altered fitness in the abse...,NaN,NaN
3,Rv0004,-,hypothetical protein,PE35_KO_vs_mbio_H37Rv,-3.81,1.00000,14,1,Mutants exhibiting altered fitness in the abse...,NaN,NaN
4,Rv0005,gyrB,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,PE35_KO_vs_mbio_H37Rv,-0.22,1.00000,14,1,Mutants exhibiting altered fitness in the abse...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
259515,Rv3921c,-,Probable conserved transmembrane protein,zhang_wt_mouse_d45_vs_zhang_input_library,0.00,1.00000,1,3,Mutants exhibiting altered fitness in mouse in...,Tryptophan biosynthesis protects mycobacteria ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...
259516,Rv3922c,-,Possible hemolysin,zhang_wt_mouse_d45_vs_zhang_input_library,-2.06,0.32788,1,3,Mutants exhibiting altered fitness in mouse in...,Tryptophan biosynthesis protects mycobacteria ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...
259517,Rv3923c,rnpA,Ribonuclease P protein component RnpA (RNaseP ...,zhang_wt_mouse_d45_vs_zhang_input_library,-3.80,0.81749,1,3,Mutants exhibiting altered fitness in mouse in...,Tryptophan biosynthesis protects mycobacteria ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...
259518,Rv3924c,rpmH,50S ribosomal protein L34 RpmH,zhang_wt_mouse_d45_vs_zhang_input_library,0.00,1.00000,1,3,Mutants exhibiting altered fitness in mouse in...,Tryptophan biosynthesis protects mycobacteria ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...


In [78]:
main_data.isna().sum()

Rv_ID                              0
gene_name                       2496
Description                     2496
Expt                               0
log2FC                          3792
q-val                           3792
num_replicates_control             0
num_replicates_experimental        0
meaning                        36495
paper_title                    60825
paper_URL                      60825
dtype: int64

In [79]:
main_data=main_data.dropna(axis=0, subset=['log2FC', 'q-val'])

In [80]:
main_data.isna().sum()

Rv_ID                              0
gene_name                        568
Description                      568
Expt                               0
log2FC                             0
q-val                              0
num_replicates_control             0
num_replicates_experimental        0
meaning                        35910
paper_title                    59850
paper_URL                      59850
dtype: int64

In [82]:
main_data.isna().sum()

Rv_ID                              0
gene_name                        568
Description                      568
Expt                               0
log2FC                             0
q-val                              0
num_replicates_control             0
num_replicates_experimental        0
meaning                        35910
paper_title                    59850
paper_URL                      59850
dtype: int64

In [85]:
main_data.loc[pd.isna(main_data['gene_name']), 'gene_name']=main_data.loc[pd.isna(main_data['gene_name']), 'Rv_ID']
main_data.loc[main_data['gene_name']=='-', 'gene_name']=main_data.loc[main_data['gene_name']=='-', 'Rv_ID']


In [86]:
main_data.isna().sum()

Rv_ID                              0
gene_name                          0
Description                      568
Expt                               0
log2FC                             0
q-val                              0
num_replicates_control             0
num_replicates_experimental        0
meaning                        35910
paper_title                    59850
paper_URL                      59850
dtype: int64

In [88]:
main_data.to_csv('../../data/data_dash_meta.csv', index=False)